In [ ]:
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
results_lu = {}
xwords

In [ ]:
import os
import importlib.util
import inspect
def getClassesFromFile(file_path):
    classes = []
    module_name = os.path.splitext(os.path.basename(file_path))[0]
    spec        = importlib.util.spec_from_file_location(module_name, file_path)
    module      = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj) and obj.__module__ == module.__name__: classes.append(obj)
    return classes
#models = ['gemma3:1b', 'gemma3:27b'] # ['gemma3:1b', 'gemma3:4b', 'phi4', 'qwq', 'gemma3:27b', 'llama3.2', 'granite3.1-dense', 'qwen2.5:14b', 'deepseek-r1:32b']
models = ['exaone-deep','cogito:3b','cogito:32b','deepcoder:1.5b','deepcoder','llama3.2:1b',
          'deepseek-r1:8b','deepseek-r1:7b','deepseek-r1:1.5b','gemma3:1b','gemma3:4b',
          'gemma3:12b','llama3.2-vision:11b','gemma3:27b','qwq',
          'phi4-mini','openthinker:32b','deepseek-r1:32b','granite3.1-dense',
          'llama3.2','phi4','qwen2.5:14b','deepseek-r1:14b',]
for _model_ in models:
    print(_model_)
    for _filename_ in os.listdir():
        if _filename_.endswith('.py') == False: continue
        for _class_ in getClassesFromFile(_filename_):
            if issubclass(_class_, XWordsSolver):
                if _class_.__name__ not in results_lu: results_lu[_class_.__name__] = {}
                print(_class_)
                if _model_ in results_lu[_class_.__name__]: continue
                xwords_copy = copy.deepcopy(xwords)
                _instance_ = _class_(xwords=xwords_copy, model=_model_)
                answer_lu, request_stats, num_of_llm_requests = _instance_.solve()
                char_level_acc = xwords_copy.characterLevelAccuracy()
                results_lu[_class_.__name__][_model_] = (answer_lu, request_stats, num_of_llm_requests, char_level_acc, xwords_copy)
                print(f'\n{_class_.__name__} {_model_} {char_level_acc}')

In [ ]:
_tiles_ = []
for _algo_ in results_lu.keys():
    for _model_ in models:
        if _model_ not in results_lu[_algo_]: _xwords_ = copy.deepcopy(xwords)
        else:                                 _xwords_ = results_lu[_algo_][_model_][4]
        _tiles_.append(_xwords_.smallMultipleSVG())
rt.table(_tiles_, per_row=len(models), spacer=10)

In [ ]:
import pandas as pd
_df_answers_ = {'algorithm':[], 'model':[], 'clue_number':[], 'orientation':[], 'answer':[]}
_df_results_ = {'algorithm':[], 'model':[], 'char_level_accuracy':[], 'num_of_llm_requests':[], 'time':[], 'prompt_tokens':[], 'output_tokens':[]}
for _algo_ in results_lu.keys():
    for _model_ in results_lu[_algo_].keys():
        _tuple_ = results_lu[_algo_][_model_]
        _time_sum_, _prompt_sum_, _output_sum_ = 0.0, 0, 0
        for x in _tuple_[1]:
            _time_sum_   += x[2]
            _prompt_sum_ += x[3]
            _output_sum_ += x[4]
        _num_of_llm_requests_ = _tuple_[2]
        _char_level_accuracy_ = _tuple_[3]
        for _clue_ in _tuple_[0].keys():
            _df_answers_['algorithm'].append(_algo_)
            _df_answers_['model'].append(_model_)
            _df_answers_['clue_number'].append(_clue_[0])
            _df_answers_['orientation'].append(_clue_[1])
            _df_answers_['answer'].append(_tuple_[0][_clue_])
        _df_results_['algorithm'].append(_algo_)
        _df_results_['model'].append(_model_)
        _df_results_['char_level_accuracy'].append(_char_level_accuracy_)
        _df_results_['num_of_llm_requests'].append(_num_of_llm_requests_)
        _df_results_['time'].append(_time_sum_)
        _df_results_['prompt_tokens'].append(_prompt_sum_)
        _df_results_['output_tokens'].append(_output_sum_)
df_answers = pd.DataFrame(_df_answers_)
df_results = pd.DataFrame(_df_results_)
#df_answers.to_parquet(_dir_ + '20250412b_xwords_answers.parquet')
#df_results.to_parquet(_dir_ + '20250412b_xwords_results.parquet')

In [ ]:
parms = {'color_by':'model', 'w':384, 'h':384}
rt.tile([rt.xy(df_results, x_field='time', y_field='char_level_accuracy', dot_size='large', **parms),
         rt.histogram(df_results, bin_by='algorithm', count_by='char_level_accuracy', h=384, w=256),
         rt.histogram(df_results, bin_by='model',     count_by='char_level_accuracy', **parms)])